In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        self.weights1 = np.random.randn(self.hidden_size, self.input_size)
        self.biases1 = np.random.randn(self.hidden_size, 1)
        self.weights2 = np.random.randn(self.output_size, self.hidden_size)
        self.biases2 = np.random.randn(self.output_size, 1)
        
    def forward(self, X, y):
        self.z1 = self.weights1 @ X.T + self.biases1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = self.weights2 @ self.a1 + self.biases2
        self.a2 = self.sigmoid(self.z2)
        print(f"MSE: {self.mse(y.T, self.a2)}")
    
    def sigmoid(self, s):
        return 1/(1+np.exp(-s))

    def sigmoid_derivative(self, s):
        return s * (1 - s)

    def backward(self, X, y):
        m = X.shape[0]
        y = y.reshape(-1, 1)
    
        self.error_delta = -2*(y.T - self.a2)
        self.z2_delta = self.sigmoid_derivative(self.a2) * self.error_delta
        self.z1_delta = self.sigmoid_derivative(self.a1) * (self.weights2.T @ self.z2_delta)

        self.weights2_delta = self.z2_delta @ self.a1.T / m
        self.biases2_delta = np.sum(self.z2_delta, axis=1, keepdims=True) / m

        self.weights1_delta = self.z1_delta @ X / m
        self.biases1_delta = np.sum(self.z1_delta, axis=1, keepdims=True) / m

        self.weights1 -= self.learning_rate*self.weights1_delta
        self.biases1 -= self.learning_rate*self.biases1_delta

        self.weights2 -= self.learning_rate*self.weights2_delta
        self.biases2 -= self.learning_rate*self.biases2_delta
    
    def mse(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)
        
    def train(self, X, y):
        self.forward(X, y)
        self.backward(X, y)

In [3]:
X = datasets.load_boston()['data']
y = datasets.load_boston()['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
self = NeuralNetwork(input_size = X_train.shape[1], hidden_size = 50, output_size = 1, learning_rate = 0.01)

In [5]:
for _ in range(10):
    self.train(X_train, y_train)

MSE: 598.2505509380618
MSE: 593.4482224229868
MSE: 585.4265404687435
MSE: 576.3651338154432
MSE: 570.2060138528817
MSE: 566.9181773364104
MSE: 565.2082929217422
MSE: 564.269357186192
MSE: 563.712205511772
MSE: 563.3560230874422
